In [1]:
import pandas as pd
import numpy as np
import implicit
import scipy.sparse as sp
from tqdm.auto import tqdm
from tqdm.contrib import tzip
import gc
from lightfm.data import Dataset
from lightfm import LightFM

In [2]:
!nvidia-smi

Tue Oct 31 21:47:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   44C    P0    72W / 300W |  11707MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#df = pd.read_csv('./vseros-rank-otb/train_df.tsv',sep='\t')
df = pd.read_parquet('all_train_df.parquet')
df

,Unnamed: 0,community_id,description,customers_count,messages_count,type,region_id,themeid,business_category,business_parent,customer_id,status,join_request_date,count_db,pair
15051500,15051500,d501adec049a83d7e968b9836d55d212e952e157fe4750...,False,6429,1,7,1.040205e+10,234.0,CARS_MACHINERY,AUTO,1a4bd984b1a1c6ef217dafdbd4c69734c2ecc785ae87a2...,I,NaN,1,1a4bd984b1a1c6ef217dafdbd4c69734c2ecc785ae87a2...
8983316,8983316,2e1b38e5b2798ad629fbcdc65e753ac381a50e7ab0b059...,False,50674,1768,7,NaN,NaN,COOKING_AND_RECIPES,FOOD,fe11157c925c62cd0538220d613a4bc450a064cb83d31a...,A,NaN,2,fe11157c925c62cd0538220d613a4bc450a064cb83d31a...
7000208,7000208,992a757cb164d3f767295af497039ad6cc3e3fa091b42a...,False,6030,1,7,NaN,NaN,COOKING_AND_RECIPES,FOOD,1bd41531bace0d2d228797f2e0bfde305ba34f7f5920a7...,A,NaN,2,1bd41531bace0d2d228797f2e0bfde305ba34f7f5920a7...
10065277,10065277,5189397d7058464aff8179b792dfda693c8cd9d09bd8b5...,True,322968,91,7,NaN,392.0,COOKING_AND_RECIPES,FOOD,944f31e05522ec9b6ae05f6acac919009af9f2e4266018...,P,NaN,1,944f31e05522ec9b6ae05f6acac919009af9f2e4266018...
13824793,13824793,56d54d70c65fae5f81f8db9267a3b58698ad18ab070831...,False,1681,2,7,1.039461e+10,NaN,OTHER_SERVICES,PROFESSIONAL_SERVICES,83d52f013852c463b478507f2397656aba59d666c3884e...,P,NaN,1,83d52f013852c463b478507f2397656aba59d666c3884e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079595,1079595,5189397d7058464aff8179b792dfda693c8cd9d09bd8b5...,True,322968,91,7,NaN,392.0,COOKING_AND_RECIPES,FOOD,d4e5e6db0cc53dcf427a3ab16d2daf5de7c8e3bfef2e51...,A,NaN,1,d4e5e6db0cc53dcf427a3ab16d2daf5de7c8e3bfef2e51...
10497335,10497335,9a227e1308c5779bba98d49fda822adf75f5e2b9ccf013...,False,1071,68,7,NaN,NaN,CITY_REGION,SOCIAL_ORGANISATIONS,2f4a89e4ae63c7bbacde5346c87f4176016c83ceb23c82...,I,NaN,1,2f4a89e4ae63c7bbacde5346c87f4176016c83ceb23c82...
7541092,7541092,e7dbc619c99a4d66f5b2e430f43e5b9f5a7d747bd74494...,True,112,2,7,1.410600e+05,NaN,RELIGIOUS_INSTITUTION,SOCIAL_ORGANISATIONS,bfc1202382dd7cf4508788c90d3f6de8cba8401da6b810...,I,NaN,1,bfc1202382dd7cf4508788c90d3f6de8cba8401da6b810...
12386537,12386537,9c8888ae8084395eead0d161b5c4c2f9ac03b9f5537fe0...,False,40613,47,7,NaN,NaN,HUMOR,BLOG,2409b4076cad4c3f6078e04120ab41cc25a9e5c9cbd680...,I,NaN,1,2409b4076cad4c3f6078e04120ab41cc25a9e5c9cbd680...


In [4]:
test_idx = np.load('./test_index.npy')
train_gbm_idx = np.load('./train_gbm_index.npy')
train_idx = list(set(df.index) - (set(test_idx) | set(train_gbm_idx)) )

In [5]:
test_df = df.loc[test_idx]
train_df_gbm = df.loc[train_gbm_idx]
train_df = df.loc[train_idx]

In [6]:
train_df['weight'] = train_df['customers_count'].map(lambda x:x**(0.33))

In [7]:
ds = Dataset()
ds.fit(train_df['customer_id'], train_df['community_id'])

In [8]:
(interactions, weights) = ds.build_interactions(train_df[['customer_id','community_id','weight']].values)

In [9]:
lightfm_mapping = ds.mapping()
lightfm_mapping = {
    'users_mapping': lightfm_mapping[0],
    'items_mapping': lightfm_mapping[2],
}

lightfm_mapping['users_inv_mapping'] = {v: k for k, v in lightfm_mapping['users_mapping'].items()}
lightfm_mapping['items_inv_mapping'] = {v: k for k, v in lightfm_mapping['items_mapping'].items()}

print(f"users_mapping amount: {len(lightfm_mapping['users_mapping'])}")
print(f"items_mapping amount: {len(lightfm_mapping['items_mapping'])}")

users_mapping amount: 1568446
items_mapping amount: 148643


In [13]:
train_df['encode_users'] = train_df['customer_id'].map(lightfm_mapping['users_mapping'].get)
train_df['encode_items'] = train_df['community_id'].map(lightfm_mapping['items_mapping'].get)

In [14]:
train_group = train_df[['encode_users','encode_items']].groupby('encode_users').agg(lambda x:x.tolist())

In [10]:
lfm_model = LightFM(
    no_components=256, # размерность пространства
    learning_rate=0.01, 
    loss='warp', 
    max_sampled=10, 
    random_state=56,
)

In [11]:
lfm_model.fit(interactions=interactions,
             epochs=10,
             num_threads=12,
             verbose=True)

Epoch: 100%|██████████| 10/10 [03:42<00:00, 22.28s/it]


In [77]:
def generate_lightfm_recs_mapper(model, item_ids, known_items,
                                 user_features, item_features, N, 
                                 user_mapping, item_inv_mapping, 
                                 num_threads=1):
    def _recs_mapper(user): # обычный юзер
        user_id = user_mapping[user] # псевдо юзер
        recs = model.predict(user_id, # Для какого юзера будем рекомендовать
                             item_ids, # массив айтемов из обучения
                             user_features=user_features, # np.float32 csr_matrix of shape [n_users, n_user_features], optional
                             item_features=item_features,  # np.float32 csr_matrix of shape [n_items, n_item_features], optional
                             num_threads=num_threads)
        
        additional_N = len(known_items.loc[user_id])
        total_N = N + additional_N
        top_cols = np.argpartition(recs, -np.arange(total_N))[-total_N:][::-1]
        top_recs = np.sort(recs)[::-1][:total_N]
        #final_recs = [item_inv_mapping[item] for item in top_cols]
        if additional_N > 0:
            filter_items = known_items.loc[user_id].values.tolist()[0]
            final_recs = [item for item in top_cols if item not in filter_items]
            top_recs = [score for item,score in zip(top_cols,top_recs) if item not in filter_items]
        final_recs = [item_inv_mapping[item] for item in final_recs]
        return final_recs[:N],top_recs[:N]
    return _recs_mapper

In [78]:
mapper = generate_lightfm_recs_mapper(
    lfm_model, 
    item_ids=train_df['encode_items'].unique(), 
    known_items=train_group, # тут можно добавить айтемы из трейна, чтобы они не повторялись в предсказаниях
    N=50,
    user_features=None, 
    item_features=None, 
    user_mapping=lightfm_mapping['users_mapping'],
    item_inv_mapping=lightfm_mapping['items_inv_mapping'],
    num_threads=1
)

In [80]:
def get_df(test_users,test_preds,scores):
    df = []
    for i,(user) in enumerate(test_users):
        for rank,(item,scor) in enumerate(zip(test_preds[i],scores[i])):
            fc = dict()
            fc['query'] = i
            fc['user_id'] = user
            fc['scor'] = scor
            fc['rank'] = rank
            fc['comunity'] = item
            df += [fc]
    return pd.DataFrame(df)

In [81]:
def set_label(df,group):
    labels = []
    for user,item in tzip(df['user_id'],df['comunity']):
        if item in group.loc[user][0]:
            labels += [1]
        else:
            labels += [0]
    df['label'] = labels
    return df

In [82]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
preds = []
for usr in tqdm(test_df['customer_id'].unique()):
    preds += [mapper(usr)]

  0%|          | 0/30000 [00:00<?, ?it/s]

In [24]:
act = test_group['community_id'].tolist()
mapk(act,test_preds,k=7)

0.0825672738095238

In [21]:
sorted_tr_data = train_df.sort_values(by='customers_count')[::-1]
best_subs = sorted_tr_data.community_id.unique()[:200]
simple_preds = best_subs[:7].tolist()
mapk(act,[simple_preds] * len(act),k=7)

0.007469630952380953

In [22]:
train_gbm_group = train_df_gbm[['community_id','customer_id']].groupby('customer_id').agg(lambda x:x.tolist())

In [23]:
gbm_preds,gbm_scores = predict_impl(model,
                                    train_gbm_group.index,
                                    train_mat,
                                    users_mapping,
                                    items_inv_mapping,
                                    N=50,
                                    falh=True)

  0%|          | 0/160000 [00:00<?, ?it/s]

In [35]:
act = train_gbm_group['community_id'].tolist()
mapk(act,gbm_preds,k=7)

0.07634596726190475

In [53]:
rank_df = get_df(train_gbm_group.index,gbm_preds,gbm_scores)

In [54]:
rank_df = set_label(rank_df,train_gbm_group)

  0%|          | 0/8000000 [00:00<?, ?it/s]

In [58]:
rank_df

,query,user_id,scor,rank,comunity,label
0,0,0000079a765e2fd7fbb3d8c6968474d7a40cc9ada4c9ec...,0.596185,0,e9f678831a65bad6bb222d9ee85e631278341739692a31...,0
1,0,0000079a765e2fd7fbb3d8c6968474d7a40cc9ada4c9ec...,0.578065,1,8ae6f3886a45df709a7a7b097d9c158ba2c5c8b50656de...,0
2,0,0000079a765e2fd7fbb3d8c6968474d7a40cc9ada4c9ec...,0.572613,2,e9ab4d850c72c29d8bc0ec6f550b9c551cc45cbc94e76a...,0
3,0,0000079a765e2fd7fbb3d8c6968474d7a40cc9ada4c9ec...,0.559779,3,9869c2a4099ce87effd0b7bbde552b05c5a5af117b51d4...,0
4,0,0000079a765e2fd7fbb3d8c6968474d7a40cc9ada4c9ec...,0.520689,4,0ef2ef940c9e91fd712307cfe31d7b4ba8143746e2f295...,0
...,...,...,...,...,...,...
7999995,159999,ffffea67894044381d713871f6cb6d482a45c031b1272f...,0.190832,45,c95cbf29e819cd3b50c38074b0116fa09d391fbbc4ab55...,0
7999996,159999,ffffea67894044381d713871f6cb6d482a45c031b1272f...,0.189663,46,4283eb74b02cb240025e2479b9c9fe8e35fdbac05c4e71...,0
7999997,159999,ffffea67894044381d713871f6cb6d482a45c031b1272f...,0.189280,47,047297bf3f2863b22a4fe96869c426f77225f4cc9987e4...,0
7999998,159999,ffffea67894044381d713871f6cb6d482a45c031b1272f...,0.186846,48,797e2de1c9b64a7b9d2e399aa68a31a0a11e203af477e7...,0


In [59]:
rank_df.to_parquet('train_gbm.parquet')

In [60]:
test_pr_df = get_df(test_group.index,test_preds,test_scores)
test_pr_df.to_parquet('test_gbm.parquet')

In [33]:
### Inference
predict_users = pd.read_csv('test_customer_ids.csv')['customer_id']

preds_users,preds_scores = predict_impl(model,
                       predict_users,
                       train_mat,
                       users_mapping,
                       items_inv_mapping,
                       N=50,
                       falh=True)

  0%|          | 0/69046 [00:00<?, ?it/s]

In [61]:
pred_pr_df = get_df(predict_users,preds_users,preds_scores)
pred_pr_df.to_parquet('pred_gbm.parquet')

In [63]:
pred_pr_df

,query,user_id,scor,rank,comunity
0,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.659503,0,768680a00db384987ff7532abc1ebc8062281b4ac60124...
1,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.453022,1,390a9c1054f80353840567b156cb0b703d3c2a963a5d70...
2,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.394046,2,6b1a156739da983b1d56d32cc8cb128b5b2a1c829de63c...
3,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.327316,3,4ec66521f946fe87d1f3d75e5ef39e01fb6b89996fbe8d...
4,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.315413,4,1bc4be66dec83175f476027b8fdd6629116600484ff948...
...,...,...,...,...,...
3452295,69045,f57918286b0047d2884e72631b65b4c7f13f5ffc697743...,0.135320,45,8747d72c9c2a844f84ca9e24a9619dae1b91a03a99a7e8...
3452296,69045,f57918286b0047d2884e72631b65b4c7f13f5ffc697743...,0.133768,46,10a7caa8692222afeae00c50672764c845594e4e067986...
3452297,69045,f57918286b0047d2884e72631b65b4c7f13f5ffc697743...,0.132896,47,9fee5e127cd5f9c760afe828abffc05361918aed2f9306...
3452298,69045,f57918286b0047d2884e72631b65b4c7f13f5ffc697743...,0.131968,48,d5cf706fd39463d378617045254f9f3d41acf3446536be...


In [43]:
def get_submit(predict_users,preds):
    df = []
    for usr,prds in zip(predict_users,preds):
        fc = dict()
        fc['customer_id'] = usr
        for i in range(7):
            fc[f'community_id_{i+1}'] = prds[i]
        df += [fc]
    return pd.DataFrame(df)

In [44]:
predict_users

0        d811a09d435ac2d1b1ed46e272405af10933b4711f4564...
1        73821118fc33500efaa6b1adf8ab0e9d314abb15f62603...
2        6381971c002097a94b8d7a03d9dc3e9ff7872a52c4764a...
3        250d49b476af0c7d2b23dd39cbb6edff39d44c64f8ebc0...
4        2da339a6bfe7791329ca8eb0a11544d4a9fb4c89572716...
                               ...                        
69041    606d6caf18a6209acaa46a1a0233a4bf22bc9885d70397...
69042    a71392c55f8bace9bd108a1acbf9c2335d700fb041966b...
69043    7b6d03389208df699aa116563dc5c45cbf5c725b47b415...
69044    8d1c564ba9f96dafb0a856fbc65fc4f9fc020863fe3b3f...
69045    f57918286b0047d2884e72631b65b4c7f13f5ffc697743...
Name: customer_id, Length: 69046, dtype: object

In [45]:
simple_preds = pd.read_csv("i_hate_als.csv")

In [48]:
submit = get_submit(predict_users,preds_users)

In [49]:
submit

,customer_id,community_id_1,community_id_2,community_id_3,community_id_4,community_id_5,community_id_6,community_id_7
0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,768680a00db384987ff7532abc1ebc8062281b4ac60124...,390a9c1054f80353840567b156cb0b703d3c2a963a5d70...,6b1a156739da983b1d56d32cc8cb128b5b2a1c829de63c...,4ec66521f946fe87d1f3d75e5ef39e01fb6b89996fbe8d...,1bc4be66dec83175f476027b8fdd6629116600484ff948...,53bbe4405c6abfe03092a641607e70637bfafdf420b088...,c295f4c57700810a3ff8fbb756c8e4a734735a2f3cc553...
1,73821118fc33500efaa6b1adf8ab0e9d314abb15f62603...,4213011a0f0e9cd636d87e9d71e01a1d9fd4e993b90b8d...,f146596214865eb9e4abf6a4d494480e4ad79a0191bd25...,b0dadd3ca6aa79e83ede40fe5144e46b04efe1fd37fb67...,630dc1e3f659d81198f8b62021224e8b8171c2f43c70d1...,6fd0b5b7561fd433e924b113a47f33e46e431a4fdb129f...,8745732c986cf22357c396bd28f36a113a401341c5d93b...,3d211ccae5e1cc0beb2d25f26c0abe1d4ad8aa6052fb8d...
2,6381971c002097a94b8d7a03d9dc3e9ff7872a52c4764a...,8460b086f0f8b7b095bee5ee8c8bb75ac68a2fae63b5cc...,47e538b3bbe82201e157853eeebb5f0c6178f11e9b9bfe...,c725659bd43a8805b7c574c1caea37e0a144069be78fa0...,312c1b76b3302377524f6b6833b6ea6257a621a5538f38...,58a725cb21fd8f0fc9d5c66cc9a6c20fa2d034a3415c9f...,0cd824b37bb5390d0e6ca98fbfd57567e91132a2ac4745...,4d6e6a696cf450cb39b8007161ea55e9c02763e71eec33...
3,250d49b476af0c7d2b23dd39cbb6edff39d44c64f8ebc0...,892f2c17a6b9684c4dfd9834c0c6c8e7694c9faa00a0b3...,a8b15b543469eebe27c67c0c5897f6dc18e160cc5997c0...,16183b4e07446d00c10d26deb9a1d62814d67fb6039b59...,7ed0a5203360da0630e5ad51f2037a4d81b97e000d1cf1...,abf30b1c3a454eaff00dea8e353bdef6c41ca9ea303ca6...,6729664ac03bd48062612647597252d30d097b79e8308b...,a8d0b95bb7330d3a1171c439c5db88e1a3671edfbca950...
4,2da339a6bfe7791329ca8eb0a11544d4a9fb4c89572716...,1717cd8dbd989b4bd56009c53db75d564b252a3f3ed722...,4d7cf04c8e56f0797012064220d0a45509a1264bc17f58...,b8ed0e918d9f11140c70ef08be19c197917f4f4455f255...,2297e8137751faa879da9ceea4a47423a9eda35d9b0278...,aee65cc087f56024df2aedd0755db2a3d193cf2ca558b7...,b5a2dec8b816fac30f1cfae978bc7afc2c24e61e05e956...,3823a5e13dfd871d3ff72d5a69af9b77466d604f43cecd...
...,...,...,...,...,...,...,...,...
69041,606d6caf18a6209acaa46a1a0233a4bf22bc9885d70397...,627c13198ee34d5075388c609a38e9192ff8a1c3acb10e...,e3adbab2d48db35c1a50f513ccc5416618fad8ef1e0567...,face1187302b0000986e09146fe04bdb5f39c4cd0ab33a...,57d99404c90c9344d3aec2c15ffb7b1a0f46c8a4f7157a...,13bc935e69c0119a69b6cb8f2a75e0150aa6c579fa826b...,99123cc6fe75a9e8485568cdcd407cf427ff33c28ab0c8...,ad51e9f700230da7c9f4bb6f3228f91f20f0fce27e5e71...
69042,a71392c55f8bace9bd108a1acbf9c2335d700fb041966b...,b2e550048439263d779622fc9b4d221827573e9c6c5900...,dac62fcdc52f97daad2712837a808e304fd011bd5ec665...,b274af0e3a0afc5445b804d67cb2e789bbfff563c4478e...,f7c22598d4d68bc4478d3b07c5f1dfee87e77bbcbf6169...,7dab772c40d0df013a66aacf7f81f8cc446d7ff50cac2d...,c3b4828f5f3954a00f233447c256ee0513a0df083ac7e9...,01f10e1db1639b341ad35be0d285ae1ffd4cc6961c0784...
69043,7b6d03389208df699aa116563dc5c45cbf5c725b47b415...,31fc05af0cdd3b70451411c633955b9baa2dfbf150d23a...,6682dfc418774213023298fe0618332b85e6f4f2095e62...,d5bf1f56574d7af705d4cd8b882ef2456cc857faa7bdfa...,7e95f5c89f6c638a50bc6f6ebd54616a85c22db0e06e6e...,ad13cacd684baf78be5f3df56634301783feb9b36a77d4...,13ac2f3d04f9306b4e3cc82e69b3f8e69f4dc7f12cf42d...,8c66ebf5e8b4e98365042fdbee3be90c2179928707c933...
69044,8d1c564ba9f96dafb0a856fbc65fc4f9fc020863fe3b3f...,311db6eae03a8ae679ec1de8c2321a120938b3eaabec93...,a33084d6d4493860f9a54b581b0e5d7b353147e8b145ff...,223f53c800c0ff65d3c72982ce6d333fcb2af30f4fbbcc...,cf6f3f535590e0ea28a73d1586a5c4ca0c30670cdf0562...,872892c975286011a5bdbf41ccbba8f1a1331a06097d23...,ca786bb0b6ccef4936b7b71c1e2aaf33b713a23fbd1f75...,12ed588a367d4c4751b906d8764b28e869e83f5d8ff2d5...


In [47]:
submit.to_csv('submit.csv',index=False)

NameError: name 'submit' is not defined

In [29]:
!zip submit.csv.zip submit.csv

updating: submit.csv (deflated 49%)


In [40]:
!unzip submit.csv.zip -y

Archive:  submit.csv.zip
caution: filename not matched:  -y


In [41]:
pd.read_csv('submit.csv')

,customer_id,community_id_1,community_id_2,community_id_3,community_id_4,community_id_5,community_id_6,community_id_7
0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,768680a00db384987ff7532abc1ebc8062281b4ac60124...,390a9c1054f80353840567b156cb0b703d3c2a963a5d70...,6b1a156739da983b1d56d32cc8cb128b5b2a1c829de63c...,1bc4be66dec83175f476027b8fdd6629116600484ff948...,53bbe4405c6abfe03092a641607e70637bfafdf420b088...,4ec66521f946fe87d1f3d75e5ef39e01fb6b89996fbe8d...,c295f4c57700810a3ff8fbb756c8e4a734735a2f3cc553...
1,73821118fc33500efaa6b1adf8ab0e9d314abb15f62603...,f146596214865eb9e4abf6a4d494480e4ad79a0191bd25...,630dc1e3f659d81198f8b62021224e8b8171c2f43c70d1...,6fd0b5b7561fd433e924b113a47f33e46e431a4fdb129f...,b0dadd3ca6aa79e83ede40fe5144e46b04efe1fd37fb67...,4213011a0f0e9cd636d87e9d71e01a1d9fd4e993b90b8d...,f9c36faa15ab170a634b9989ee84fb2f76f185273ee254...,870363f525d387304453fca7384f7fd9bd0394efb99fa7...
2,6381971c002097a94b8d7a03d9dc3e9ff7872a52c4764a...,8460b086f0f8b7b095bee5ee8c8bb75ac68a2fae63b5cc...,c725659bd43a8805b7c574c1caea37e0a144069be78fa0...,58a725cb21fd8f0fc9d5c66cc9a6c20fa2d034a3415c9f...,0cd824b37bb5390d0e6ca98fbfd57567e91132a2ac4745...,312c1b76b3302377524f6b6833b6ea6257a621a5538f38...,0ad25a8e972a9467effdf4bac9478588df77814268cf69...,6db8319fff1210bdb148875123407620c5f364eacb3ffe...
3,250d49b476af0c7d2b23dd39cbb6edff39d44c64f8ebc0...,892f2c17a6b9684c4dfd9834c0c6c8e7694c9faa00a0b3...,a8b15b543469eebe27c67c0c5897f6dc18e160cc5997c0...,16183b4e07446d00c10d26deb9a1d62814d67fb6039b59...,7ed0a5203360da0630e5ad51f2037a4d81b97e000d1cf1...,abf30b1c3a454eaff00dea8e353bdef6c41ca9ea303ca6...,6729664ac03bd48062612647597252d30d097b79e8308b...,a8d0b95bb7330d3a1171c439c5db88e1a3671edfbca950...
4,2da339a6bfe7791329ca8eb0a11544d4a9fb4c89572716...,1717cd8dbd989b4bd56009c53db75d564b252a3f3ed722...,b8ed0e918d9f11140c70ef08be19c197917f4f4455f255...,4d7cf04c8e56f0797012064220d0a45509a1264bc17f58...,3823a5e13dfd871d3ff72d5a69af9b77466d604f43cecd...,2297e8137751faa879da9ceea4a47423a9eda35d9b0278...,b4823c708c5fdba713f8f150327ce2101a5061d4f9d1f7...,965a4548932aa04bf2c7f34be8e328aaa58edd31bc26b6...
...,...,...,...,...,...,...,...,...
69041,606d6caf18a6209acaa46a1a0233a4bf22bc9885d70397...,627c13198ee34d5075388c609a38e9192ff8a1c3acb10e...,e3adbab2d48db35c1a50f513ccc5416618fad8ef1e0567...,57d99404c90c9344d3aec2c15ffb7b1a0f46c8a4f7157a...,face1187302b0000986e09146fe04bdb5f39c4cd0ab33a...,13bc935e69c0119a69b6cb8f2a75e0150aa6c579fa826b...,99123cc6fe75a9e8485568cdcd407cf427ff33c28ab0c8...,0da0c2796eeeb6442948813494857eec686e46897412b8...
69042,a71392c55f8bace9bd108a1acbf9c2335d700fb041966b...,b2e550048439263d779622fc9b4d221827573e9c6c5900...,b274af0e3a0afc5445b804d67cb2e789bbfff563c4478e...,dac62fcdc52f97daad2712837a808e304fd011bd5ec665...,f7c22598d4d68bc4478d3b07c5f1dfee87e77bbcbf6169...,7dab772c40d0df013a66aacf7f81f8cc446d7ff50cac2d...,c3b4828f5f3954a00f233447c256ee0513a0df083ac7e9...,3c8c56c125be2575c74750926654d11289941ff9b3f75e...
69043,7b6d03389208df699aa116563dc5c45cbf5c725b47b415...,31fc05af0cdd3b70451411c633955b9baa2dfbf150d23a...,6682dfc418774213023298fe0618332b85e6f4f2095e62...,13ac2f3d04f9306b4e3cc82e69b3f8e69f4dc7f12cf42d...,d5bf1f56574d7af705d4cd8b882ef2456cc857faa7bdfa...,ad13cacd684baf78be5f3df56634301783feb9b36a77d4...,7e95f5c89f6c638a50bc6f6ebd54616a85c22db0e06e6e...,8c66ebf5e8b4e98365042fdbee3be90c2179928707c933...
69044,8d1c564ba9f96dafb0a856fbc65fc4f9fc020863fe3b3f...,311db6eae03a8ae679ec1de8c2321a120938b3eaabec93...,872892c975286011a5bdbf41ccbba8f1a1331a06097d23...,7dde421f7b79b30c5a575162421f5540a190be98ade166...,ca786bb0b6ccef4936b7b71c1e2aaf33b713a23fbd1f75...,cf6f3f535590e0ea28a73d1586a5c4ca0c30670cdf0562...,4cd507db4bc028745427889e579bbf3591662d2885a088...,ff5b9c28068f1ffe93af9c29abda06a7e6fc9fb4eda177...
